In [8]:
!pip install wget pillow piexif geopy # Added wget to the installation command

  Preparing metadata (setup.py) ... done
  Created wheel for wget: filename=wget-3.2-py3-none-any.whl size=9656 sha256=4642786697ef72a9f6059ee188515467bbfb1a783a28650b4ec95d93972924b0
  Stored in directory: /root/.cache/pip/wheels/8b/f1/7f/5c94f0a7a505ca1c81cd1d9208ae2064675d97582078e6c769
Successfully built wget


In [14]:
import os
from PIL import Image, ImageDraw, ImageFont
import piexif
from geopy.geocoders import Nominatim
from geopy.exc import GeocoderTimedOut

# Configuration
INPUT_FOLDER = '/content/'              # Set to your input folder path
OUTPUT_FOLDER = '/content/output'       # Set to your output folder path
FONT_PATH = '/content/fonts/LiberationSans-Regular.ttf'  # Path to your downloaded .ttf font file
FONT_SIZE = 36
TEXT_COLOR = (255, 255, 255)            # White text
TEXT_BG_COLOR = (0, 0, 0, 128)          # Semi-transparent black background
MAX_WIDTH = 800                         # Maximum width for images (optional)

# Supported image extensions
SUPPORTED_EXTENSIONS = ('.jpg', '.jpeg', '.png', '.tiff', '.bmp', '.gif')

# Initialize geolocator
geolocator = Nominatim(user_agent="image_metadata_overlay")

# Determine the appropriate resampling filter
try:
    Resampling = Image.Resampling  # Pillow >= 10
    RESAMPLE_FILTER = Resampling.LANCZOS
except AttributeError:
    RESAMPLE_FILTER = Image.LANCZOS  # Pillow < 10

def get_exif_data(image_path):
    try:
        exif_dict = piexif.load(image_path)
        return exif_dict
    except Exception as e:
        print(f"Error reading EXIF data from {image_path}: {e}")
        return None

def get_date(exif_dict):
    # Try different EXIF tags for date
    for tag in ['DateTimeOriginal', 'DateTime', 'DateTimeDigitized']:
        tag_id = getattr(piexif.ExifIFD, tag, None)
        if tag_id:
            date = exif_dict['Exif'].get(tag_id)
            if date:
                return date.decode('utf-8')
    return "Unknown Date"

def get_gps_coords(exif_dict):
    gps_info = exif_dict.get('GPS', {})
    if not gps_info:
        return None

    def _convert_to_degrees(value):
        d, m, s = value
        return d[0]/d[1] + (m[0]/m[1])/60 + (s[0]/s[1])/3600

    try:
        lat = _convert_to_degrees(gps_info[piexif.GPSIFD.GPSLatitude])
        if gps_info.get(piexif.GPSIFD.GPSLatitudeRef, b'N') != b'N':
            lat = -lat

        lon = _convert_to_degrees(gps_info[piexif.GPSIFD.GPSLongitude])
        if gps_info.get(piexif.GPSIFD.GPSLongitudeRef, b'E') != b'E':
            lon = -lon

        return (lat, lon)
    except KeyError as e:
        print(f"Missing GPS key: {e}")
        return None
    except Exception as e:
        print(f"Error converting GPS data: {e}")
        return None

def reverse_geocode(coords):
    try:
        location = geolocator.reverse(coords, timeout=10)
        if location:
            return location.address
        else:
            return "Unknown Location"
    except GeocoderTimedOut:
        print("Geocoder service timed out.")
        return "Location Unavailable"
    except Exception as e:
        print(f"Geocoding error: {e}")
        return "Location Unavailable"

def add_overlay(image, text, font_path, font_size, text_color, bg_color):
    draw = ImageDraw.Draw(image, "RGBA")
    try:
        font = ImageFont.truetype(font_path, font_size)
    except IOError:
        print(f"Font not found at {font_path}. Using default font.")
        font = ImageFont.load_default()

    text_lines = text.split('\n')
    padding = 10
    x, y = padding, padding

    # Calculate text dimensions
    text_width = 0
    total_text_height = 0
    line_heights = []
    for line in text_lines:
        bbox = draw.textbbox((0,0), line, font=font)
        line_width = bbox[2] - bbox[0]
        line_height = bbox[3] - bbox[1]
        text_width = max(text_width, line_width)
        line_heights.append(line_height)
        total_text_height += line_height

    # Calculate total height including padding between lines
    total_height = total_text_height + padding * (len(text_lines) + 1)

    # Draw background rectangle
    draw.rectangle(
        [x - padding, y - padding, x + text_width + padding, y + total_height],
        fill=bg_color
    )

    # Draw each line of text
    current_y = y
    for idx, line in enumerate(text_lines):
        draw.text((x, current_y), line, font=font, fill=text_color)
        current_y += line_heights[idx] + padding // 2  # Adjust line spacing

def process_image(image_path, output_path):
    exif_data = get_exif_data(image_path)
    if not exif_data:
        print(f"No EXIF data for {image_path}. Skipping.")
        return

    date = get_date(exif_data)
    coords = get_gps_coords(exif_data)
    location = "Unknown Location"

    if coords:
        location = reverse_geocode(coords)

    text = f"Date: {date}\nLocation: {location}"

    try:
        with Image.open(image_path) as img:
            # Optionally resize image if it's too large
            if img.width > MAX_WIDTH:
                ratio = MAX_WIDTH / float(img.width)
                new_height = int(float(img.height) * ratio)
                img = img.resize((MAX_WIDTH, new_height), RESAMPLE_FILTER)

            add_overlay(img, text, FONT_PATH, FONT_SIZE, TEXT_COLOR, TEXT_BG_COLOR)

            # Ensure output directory exists
            os.makedirs(os.path.dirname(output_path), exist_ok=True)

            # Preserve original EXIF data when saving
            if "exif" in img.info:
                img.save(output_path, exif=img.info["exif"])
            else:
                img.save(output_path)

            print(f"Processed and saved: {output_path}")
    except Exception as e:
        print(f"Error processing {image_path}: {e}")

def main():
    if not os.path.exists(INPUT_FOLDER):
        print(f"Input folder '{INPUT_FOLDER}' does not exist.")
        return

    os.makedirs(OUTPUT_FOLDER, exist_ok=True)

    for root, dirs, files in os.walk(INPUT_FOLDER):
        for file in files:
            if file.lower().endswith(SUPPORTED_EXTENSIONS):
                input_path = os.path.join(root, file)
                # Maintain directory structure in output folder
                relative_path = os.path.relpath(input_path, INPUT_FOLDER)
                output_path = os.path.join(OUTPUT_FOLDER, relative_path)
                output_dir = os.path.dirname(output_path)
                os.makedirs(output_dir, exist_ok=True)

                process_image(input_path, output_path)

if __name__ == "__main__":
    main()

Processed and saved: /content/output/IMG_20241007_122538.jpg
Processed and saved: /content/output/output/IMG_20241007_122538.jpg
